In [2]:
import os

In [24]:
os.chdir('/Users/chloehe/Desktop')

In [25]:
from keras.preprocessing.image import ImageDataGenerator

In [26]:
datagen = ImageDataGenerator(
      rotation_range = 60,
      width_shift_range = 0.1,
      height_shift_range = 0.2,
      shear_range = 0.3,
      zoom_range = 0.2,
      horizontal_flip = True,
      fill_mode = 'nearest')

In [27]:
covid_dir = '/Users/chloehe/Desktop/images'

covid_generator = datagen.flow_from_directory(
        covid_dir,
        target_size=(61, 115),
        batch_size=10,
        class_mode='binary')

Found 947 images belonging to 2 classes.


In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
import glob
from PIL import Image

In [40]:
covid_list = os.listdir('/Users/chloehe/Desktop/images/CT_COVID')
nocovid_list = os.listdir('/Users/chloehe/Desktop/images/CT_NonCOVID')

In [41]:
# read in covid CT
covid_list = []
for filename in glob.glob('CT_COVID/*.png'): 
    im=plt.imread(filename)
    covid_list.append(im)

In [33]:
# convert "jpg" to "png" and “save-as”
for filename in glob.glob('CT_NonCOVID/*.jpg'):  
    im1 = Image.open(filename)
    prefix = filename.split(".jpg")[0]
    im1.save(prefix+'.png')

In [34]:
# read in non-covid CT
nocovid_list = []
for filename in glob.glob('CT_NonCOVID/*.png'): 
    im=plt.imread(filename,0)
    nocovid_list.append(im)

In [36]:
# Find the minimum dimension for covid pictures to crop later
a = np.empty(349, dtype=object) 
for i in range(len(covid_list)):
     a[i] = covid_list[i].shape[0]
        
b = np.empty(349, dtype=object) 
for i in range(len(covid_list)):
     b[i] = covid_list[i].shape[1]
        
# min_length = min(a)
# min_width = min(b)
# print("the minimum length for covid pictures is",min_length)
# print("the minimum width for covid pictures is",min_width)

In [38]:
# Find the minimum dimension for non-covid picturesto crop later
c = np.empty(397, dtype=object) 
for i in range(len(nocovid_list)):
     c[i] = nocovid_list[i].shape[0]
        
d = np.empty(397, dtype=object) 
for i in range(len(nocovid_list)):
     d[i] = nocovid_list[i].shape[1]
        
# min_length_non_covid = min(c)
# min_width_non_covid = min(d)
# print("the minimum length for non-covid pictures is",min_length_non_covid)
# print("the minimum width for non-covid pictures is",min_width_non_covid)

In [39]:
covid_processed = []
for i in range(len(covid_list)):
    if len(covid_list[i].shape)==3:
        grey = rgb2gray(covid_list[i])
    else:
        grey=covid_list[i]
    #additional step: rotate matrix such that dim(row)<=dim(column) -> cropping causes less information loss
    if grey.shape[0]>grey.shape[1]:
        grey=np.rot90(grey)
    c = cropR(grey,min_length,min_width) 
    g = grad(c,min_length-2,min_width-2,False)
    h = hog(g[0],g[1],16,16,32)
    covid_processed.append(h)
covid_processed = np.array(covid_processed)
covid_processed.shape

(0,)